In [3]:
# Energy: XOM, CVX;
# Financial: JPM, BRK.A
# Consumer: AMZN, MCD
# Technology: AAPL, MSFT, TSLA
# Communication: META, GOOG



In [4]:
import yfinance as yf
yf.pdr_override()
from inspect import trace
import pandas as pd
import numpy as np
from pandas_datareader import data as wb
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from tqdm.auto import tqdm


tickers = ['XOM', 'CVX', 'JPM', 'BRK-A', 'AMZN', 'MCD', 'AAPL', 'MSFT', 'TSLA', 'META', 'GOOG']
data = wb.DataReader(tickers, data_source='yahoo', interval='1d')['Adj Close']
data = data.dropna()
# date only keep year and month
data.index = data.index.strftime('%Y-%m')
data.head()


[*********************100%***********************]  11 of 11 completed


,AAPL,AMZN,BRK-A,CVX,GOOG,JPM,MCD,META,MSFT,TSLA,XOM
Date,,,,,,,,,,,
2012-05,16.122007,10.6925,119850.0,63.480202,14.953949,24.797680,66.271309,38.230000,23.721563,1.837333,52.266579
2012-05,17.061275,10.9055,120220.0,64.273209,15.295419,24.072041,67.318657,34.029999,24.110575,1.918000,52.632244
2012-05,16.930264,10.7665,120550.0,64.034660,14.963912,25.182709,67.370285,31.000000,24.118681,2.053333,52.574520
2012-05,17.343365,10.8640,120455.0,63.828342,15.179603,25.367821,67.473572,32.000000,23.591890,2.068000,52.632244
2012-05,17.184084,10.7620,120217.0,64.511772,15.035145,25.153091,67.510422,33.029999,23.559475,2.018667,52.997913


In [5]:
# past five years only
data = data.loc['2018-03':]

# index to columns
data = data.reset_index()
data.to_csv('price.csv', index=False)

In [6]:
IXIC = wb.DataReader('QQQ', data_source='yahoo', interval='1d')['Adj Close']
IXIC = IXIC.loc['2018-03-13':]


[*********************100%***********************]  1 of 1 completed


In [7]:
# five years log return comparison

Rel_Perf = []
Log_Return = []
for tick in tickers:
    total_log_return_tick = np.log(data[tick]/data[tick].shift(1)).sum()
    Log_Return.append(total_log_return_tick)
    total_log_return_ixic = np.log(IXIC/IXIC.shift(1)).sum()
    # avg of diff
    Rel_Perf.append((total_log_return_tick - total_log_return_ixic)*100)
    

In [8]:
# make another dataframe
# columns: tickers, category, Market Cap, Total log return of last year, location, relative percentage performance to the Nasdaq Composite in the past 1 year

df = pd.DataFrame(columns=['Ticker', 'Category', 'Market_Cap_B', '5Y Log Return', 'Headquarter', 'Rel_Perf_Pct'])
df['Ticker'] = tickers
df['Category'] = ['Energy', 'Energy', 'Financial', 'Financial', 'Consumer', 'Consumer', 'Technology', 'Technology', 'Technology', 'Communication', 'Communication']

Location = ['TX', 'CA', 'NY', 'NE', 'WA', 'IL', 'CA', ' WA', 'CA', 'CA', 'CA']
df['Headquarter'] = Location

mkt_cap_til_031323 = [	
'433.72', '302.61','386.32', '663.62', '947.15', '192.44', '2380', '1890', '546.73', '469.01', '1170']
df['Market_Cap_B'] = mkt_cap_til_031323

df['5Y Log Return'] = Log_Return
df['Rel_Perf_Pct'] = Rel_Perf


In [9]:
df.to_csv('company_info.csv', index=False)